# Capstone Project Brief

This Capstone Project is part of the <u>5-day Gen AI Intensive Course with Google</u>, which took place from Monday March 31 - Friday April 4, 2025. This project seek to apply what we learned from the course to a real world use case/problem, and achive the following Gen AI capabilities:

- Structured output/JSON mode/controlled generation (?)
- Few-shot prompting
- Document understanding
- Image understanding
- Function Calling (?)
- Agents (?)
- Long context window
- Gen AI evaluation (?)
- Grounding (?)
- Embeddings
- Retrieval augmented generation (RAG)
- Vector search/vector store/vector database

In addition to this public Kaggle notebook, this project also included the optional submission items below:

- a blogpost: a public blogpost on the capstone project, explaining the use case, the problem(s), and the solution.
- a YouTube video: a public YouTube video on your capstone project. No minimum or maximum duration.

## Project Overview and Problem Statement

## Proposed Solution

## Project Setup

First, install ChromaDB and the Gemini API Python SDK for RAG and LLM access.
Also setup the API Key.

In [1]:
!pip uninstall -qqy jupyterlab kfp  # Remove unused conflicting packages
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"

In [2]:
!pip install python-frontmatter # Library to process markdown frontmatter for metadata

In [3]:
from google import genai
from google.genai import types

from IPython.display import Markdown

genai.__version__

'1.7.0'

In [4]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

## Explore available models

### Embedding Model

We will be using the [`embedContent`](https://ai.google.dev/api/embeddings#method:-models.embedcontent) API method to calculate embeddings of documents before saving to the database. 

- `text-embedding-004` is the most recent generally-available embedding model
- experimental `gemini-embedding-exp-03-07` model
- more info: [`models.list`](https://ai.google.dev/api/models#method:-models.list) / [the models page](https://ai.google.dev/gemini-api/docs/models/gemini#text-embedding).

### LLM Model
We will also list all generation models below and check their capabilities.

In [5]:
client = genai.Client(api_key=GOOGLE_API_KEY)

for m in client.models.list():
    if "embedContent" in m.supported_actions:
        print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


In [6]:
# Collect rows of model info
rows = [
    "| Model | Supported Actions |",
    "|-------|-------------------|"
]

for m in client.models.list():
    name = m.name
    actions = m.supported_actions

    is_embedding = "embedContent" in actions

    # Skip embedding models if desired
    if not is_embedding:
        actions_str = ", ".join(actions)
        row = f"| `{name}` | {actions_str} |"
        rows.append(row)

# Display the markdown table
display(Markdown("\n".join(rows)))

| Model | Supported Actions |
|-------|-------------------|
| `models/chat-bison-001` | generateMessage, countMessageTokens |
| `models/text-bison-001` | generateText, countTextTokens, createTunedTextModel |
| `models/embedding-gecko-001` | embedText, countTextTokens |
| `models/gemini-1.0-pro-vision-latest` | generateContent, countTokens |
| `models/gemini-pro-vision` | generateContent, countTokens |
| `models/gemini-1.5-pro-latest` | generateContent, countTokens |
| `models/gemini-1.5-pro-001` | generateContent, countTokens, createCachedContent |
| `models/gemini-1.5-pro-002` | generateContent, countTokens, createCachedContent |
| `models/gemini-1.5-pro` | generateContent, countTokens |
| `models/gemini-1.5-flash-latest` | generateContent, countTokens |
| `models/gemini-1.5-flash-001` | generateContent, countTokens, createCachedContent |
| `models/gemini-1.5-flash-001-tuning` | generateContent, countTokens, createTunedModel |
| `models/gemini-1.5-flash` | generateContent, countTokens |
| `models/gemini-1.5-flash-002` | generateContent, countTokens, createCachedContent |
| `models/gemini-1.5-flash-8b` | createCachedContent, generateContent, countTokens |
| `models/gemini-1.5-flash-8b-001` | createCachedContent, generateContent, countTokens |
| `models/gemini-1.5-flash-8b-latest` | createCachedContent, generateContent, countTokens |
| `models/gemini-1.5-flash-8b-exp-0827` | generateContent, countTokens |
| `models/gemini-1.5-flash-8b-exp-0924` | generateContent, countTokens |
| `models/gemini-2.5-pro-exp-03-25` | generateContent, countTokens |
| `models/gemini-2.5-pro-preview-03-25` | generateContent, countTokens |
| `models/gemini-2.0-flash-exp` | generateContent, countTokens, bidiGenerateContent |
| `models/gemini-2.0-flash` | generateContent, countTokens |
| `models/gemini-2.0-flash-001` | generateContent, countTokens |
| `models/gemini-2.0-flash-exp-image-generation` | generateContent, countTokens, bidiGenerateContent |
| `models/gemini-2.0-flash-lite-001` | generateContent, countTokens |
| `models/gemini-2.0-flash-lite` | generateContent, countTokens |
| `models/gemini-2.0-flash-lite-preview-02-05` | generateContent, countTokens |
| `models/gemini-2.0-flash-lite-preview` | generateContent, countTokens |
| `models/gemini-2.0-pro-exp` | generateContent, countTokens |
| `models/gemini-2.0-pro-exp-02-05` | generateContent, countTokens |
| `models/gemini-exp-1206` | generateContent, countTokens |
| `models/gemini-2.0-flash-thinking-exp-01-21` | generateContent, countTokens |
| `models/gemini-2.0-flash-thinking-exp` | generateContent, countTokens |
| `models/gemini-2.0-flash-thinking-exp-1219` | generateContent, countTokens |
| `models/learnlm-1.5-pro-experimental` | generateContent, countTokens |
| `models/gemma-3-1b-it` | generateContent, countTokens |
| `models/gemma-3-4b-it` | generateContent, countTokens |
| `models/gemma-3-12b-it` | generateContent, countTokens |
| `models/gemma-3-27b-it` | generateContent, countTokens |
| `models/aqa` | generateAnswer |
| `models/imagen-3.0-generate-002` | predict |
| `models/gemini-2.0-flash-live-001` | bidiGenerateContent, countTokens |

## Data Preparation

Loading a list of documents we can use to create an embedding database.

### Public Gensler Thought Leadership
Here we are using a list of markdown articles sourced from Gensler.com dialogue blog 2025<br>
https://www.kaggle.com/datasets/junwangzero/public-gensler-thought-leadership

In [7]:
import os

input_path = "/kaggle/input"
all_files = []

# Collect all file paths from all attached datasets
for dataset in os.listdir(input_path):
    dataset_path = os.path.join(input_path, dataset)
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            all_files.append(os.path.join(root, file))

# Print total count and the list
print(f"Total number of files: {len(all_files)}\n")
for file_path in all_files:
    print(file_path)
Markdown('---')

Total number of files: 20

/kaggle/input/public-gensler-thought-leadership/The New Experiential Hybrid.md
/kaggle/input/public-gensler-thought-leadership/The Biggest Challenge to Office Conversions Isnt Design  Its the Status Quo.md
/kaggle/input/public-gensler-thought-leadership/Realizing the Value of Artificial Intelligence When Planning Tomorrows Healthcare Facilities.md
/kaggle/input/public-gensler-thought-leadership/Prototyping the Hospital of the Future.md
/kaggle/input/public-gensler-thought-leadership/Severe Weather The New Design Imperative.md
/kaggle/input/public-gensler-thought-leadership/The Uncertainty of Electric Power A View from Houston.md
/kaggle/input/public-gensler-thought-leadership/Biodiversity as the New Frontier for Achieving Resilience in Real Estate.md
/kaggle/input/public-gensler-thought-leadership/Avoiding Fear-Based Decision-Making to Foster Positive In-Store Retail Experiences.md
/kaggle/input/public-gensler-thought-leadership/Live Event Amenities The Next 

---

We can visualize the format of an example of such article below:

In [8]:
# helper function to parse input markdown files
def read_markdown(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()

# Read all markdown files into a list of strings
documents = [read_markdown(path) for path in all_files]

print(documents[0][:800])  # Preview first 500 characters of the first doc
Markdown('---')

---
title: "The New Experiential Hybrid"
source: "https://www.gensler.com/blog/the-new-experiential-hybrid"
author:
  - "[[Gensler]]"
published:
created: 2025-04-09
description: "Gensler is a global architecture, design, and planning firm with 57 offices and 6,000+ professionals across the Americas, Europe, Greater China, and APME."
tags:
  - "clippings"
---
![A street with cars and buildings.](https://static2.gensler.com/uploads/image/96476/Sportsmens_Lodge_N7_1738969737.jpg)

The Sportsmen’s Lodge, Los Angeles, California

The convergence of our physical and digital ecosystems continues to revolutionize our lives, reshape our behaviors, and reimagine our ability to navigate the complex beauty and challenges of life in a boundaryless world. Radical technological innovations have resulted 


---

Next we will process the articles via extract its metadata from the frontmatter and do some cleaning:

- extract metadata for use in the vector database
- replace the frontmatter with a clean table
- get Gen AI summary of the article
- get Gen AI generated "reverse prompt" that could've generated this article for fine-tuning or few shot prompting purpose
- etc.

In [9]:
from google.api_core import retry

# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

In [10]:
import pprint
import frontmatter

# Helper function to translate metadata to a table
def metadata_to_table(metadata):
    lines = ["| Metadata | Value |", "|-----|-------|"]
    for key, value in metadata.items():
        lines.append(f"| {key} | {repr(value)} |")
    return '\n'.join(lines)

metadatas = []
parsed_docs = []

# Extract metadata in the original document and turn into clean table format
for doc in documents:
    post = frontmatter.loads(doc)
    metadata = post.metadata
    metadatas.append(metadata)
    table = metadata_to_table(metadata)
    title = metadata.get('title', 'Untitled')
    parsed = f"{table}\n\n# {title}\n{post.content}"
    parsed_docs.append(parsed)

display(Markdown(parsed_docs[0]))  # Preview an example of a parsed 
Markdown('---')

| Metadata | Value |
|-----|-------|
| title | 'The New Experiential Hybrid' |
| source | 'https://www.gensler.com/blog/the-new-experiential-hybrid' |
| author | ['[[Gensler]]'] |
| published | None |
| created | datetime.date(2025, 4, 9) |
| description | 'Gensler is a global architecture, design, and planning firm with 57 offices and 6,000+ professionals across the Americas, Europe, Greater China, and APME.' |
| tags | ['clippings'] |

# The New Experiential Hybrid
![A street with cars and buildings.](https://static2.gensler.com/uploads/image/96476/Sportsmens_Lodge_N7_1738969737.jpg)

The Sportsmen’s Lodge, Los Angeles, California

The convergence of our physical and digital ecosystems continues to revolutionize our lives, reshape our behaviors, and reimagine our ability to navigate the complex beauty and challenges of life in a boundaryless world. Radical technological innovations have resulted in a fundamental mindset shift of our behaviors, needs, and desires, moving from Live, Work, Play (and Shop), to a hybrid of **Live, Live, Live**.

While many of us feel Covid-19 is largely in the rearview mirror, unfortunately a tapestry of global social epidemics still affects us all. Americans are lonelier than ever: [according to the Cigna U.S. Loneliness Index](https://newsroom.thecignagroup.com/loneliness-epidemic-persists-post-pandemic-look), 58% of adults, and 79% of those aged 18-24 report being lonely. Our social networks are a third smaller now and we’re far more likely to live alone; today, nearly 28% of U.S. homes have just one occupant, up from 17% in 1969 according to the US Census. The annual number of marriages is in decline, and birth rates are slowing. In a lonelier, more divided world, the need for community is greater than ever.

There’s a lot to consider with this cascade of change occurring at the speed of culture. As landlords, developers, and designers collaborate for a better future, we need to start thinking differently. It is not enough to simply evolve the concept of our built environment. Since we now live a hybridized life, we need to make a transformative shift to a “New Hybrid” experience, one that can play a significant role in restoring our sense of closeness in our communities. The New Hybrid experience is a model that uses an integrative approach to the disciplines of art, science, and technology, helping us to develop a deeper understanding of the complexities of our modern world and guide us to produce positive and impactful outcomes.

The New Hybrid experience brings together the robust attributes of commerce, culture, and leisure, and recalibrates the urban experience. It is amenity rich, leverages technology, connects easily to transit, and offers world-class leisure and event activities. It promotes local artisans and culture, reflects generational and community values, and builds traditions.

We relate to urban environments in an emotional dimension. Signature characteristics of hybrid places — otherwise called mixed use — are personable and human-centric. The value proposition is when a space, a place, a building, or an event **moves** us. When this happens, it elicits a sense of awe and wonder, engages all of our senses, dissolves barriers, and eliminates pain points.

The emotional exchange between the built environment and the user is about attaining and retaining a person’s attention. It’s that visceral moment when something inside you connects with the place in a special way, the beyond ordinary life experience that can elicit an emotional response that requires no specialized knowledge, nothing more than one’s emotion to make it happen. And it happens to everyone. Great places have a defined persona, which has the ability to trigger aesthetic emotions — that highly emotive sense you get, which can sometimes be difficult to describe, but you just know it just feels amazing.

Here are some examples of how the New Hybrid is playing out in our project work across multiple scales:

For [The Hub on Causeway](https://www.gensler.com/projects/the-hub-on-causeway), a dynamic, 2-million-square-foot mixed-use development at Boston’s TD Garden, Gensler created a marquee sports and entertainment destination with office, residential, hotel, retail, entertainment, dining, and creative office space that culminates in a grand entrance that funnels people to TD Garden and the North Station transit hub.

![A street with people and buildings along it.](https://static2.gensler.com/uploads/image/96477/HubonCauseway_N38_1738969781.jpg)

The Hub on Causeway, Boston, Massachusetts. Photo by Anton Grassl.

On a smaller scale, Gensler transformed a former meeting and convention space into the Sportsmen’s Lodge, an upscale outdoor lifestyle center in LA’s Studio City with 95,700-square-feet of walkable retail, restaurants, fitness, and public space. An open-air plaza features communal spaces for visitors to dine or relax, while a waterfront deck hosts tenant and center events.

![People walking on a sidewalk.](https://static2.gensler.com/uploads/image/96478/Sportsmens_Lodge_N4_1738969821.jpg)

The Sportsmen’s Lodge, Los Angeles, California

**Designed for maximum happiness, the New Hybrid is a magnetic and transformational place that transcends the predictable**. An amazing hybrid experience creates cultural capital by truly caring for us and improves the way we are. If our lives are truly enriched, the place will be more than real estate — it creates a deep, emotional connection with us and truly becomes an integral part of our lives.

For media inquiries, email [media@gensler.com](https://www.gensler.com/blog/).

---

Next we can generate the summary and reverse prompt for each article and add it to the metadata list.

In [11]:
summary = 'Summarize the article in one sentence clearly.'

reverse_prompt = """\
Given this article, reverse engineer it into a one sentence prompt that would have generate this article. 
Keep the sentence clear, similar to the structure below:
"Generate an (thought leadership/research insight/etc.) article on (types of project/industry/etc.) (in relation to /regarding/etc.) (topics/issues/etc.)"
"""

# Helper Function to process article through the LLM to generate responses
@retry.Retry(predicate=is_retriable)
def summarise_doc(request: str, document_file: str) -> str:
  """Execute the request on the uploaded document."""
  # Set the temperature low to stabilise the output.
  config = types.GenerateContentConfig(temperature=0.0)
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=config,
      contents=[request, document_file],
  )
  return response.text.strip() # Remove \n and other white spaces

# Add summary to metadata
for i, doc in enumerate(parsed_docs):
    metadatas[i]['summary'] = summarise_doc(summary, doc)
    metadatas[i]['prompt'] = summarise_doc(reverse_prompt, doc)

display(Markdown(metadata_to_table(metadatas[0])))  # Preview metadata of the first doc in table format
Markdown('---')

| Metadata | Value |
|-----|-------|
| title | 'The New Experiential Hybrid' |
| source | 'https://www.gensler.com/blog/the-new-experiential-hybrid' |
| author | ['[[Gensler]]'] |
| published | None |
| created | datetime.date(2025, 4, 9) |
| description | 'Gensler is a global architecture, design, and planning firm with 57 offices and 6,000+ professionals across the Americas, Europe, Greater China, and APME.' |
| tags | ['clippings'] |
| summary | 'To combat increasing loneliness and social division, the "New Experiential Hybrid" reimagines urban spaces by integrating commerce, culture, and leisure with technology and community values to create emotionally resonant environments that foster connection and improve well-being.' |
| prompt | 'Generate a thought leadership article on the future of urban development regarding the "New Experiential Hybrid," a mixed-use approach that integrates commerce, culture, and leisure to address loneliness and build community in a rapidly changing world.' |

---

We noticed that some metadata are in list format which is not acceptable by ChromaDB, so we will do simply join the list together into a single string.

In [12]:
def clean_metadata(meta: dict) -> dict:
    return {
        k: ', '.join(v) if isinstance(v, list) else v
        for k, v in meta.items()
        if isinstance(v, (str, int, float, bool, list))  # skip unacctapble types
    }

metadatas = [clean_metadata(metadata) for metadata in metadatas]
display(Markdown(metadata_to_table(metadatas[0])))  # Preview metadata of the first doc in table format

| Metadata | Value |
|-----|-------|
| title | 'The New Experiential Hybrid' |
| source | 'https://www.gensler.com/blog/the-new-experiential-hybrid' |
| author | '[[Gensler]]' |
| description | 'Gensler is a global architecture, design, and planning firm with 57 offices and 6,000+ professionals across the Americas, Europe, Greater China, and APME.' |
| tags | 'clippings' |
| summary | 'To combat increasing loneliness and social division, the "New Experiential Hybrid" reimagines urban spaces by integrating commerce, culture, and leisure with technology and community values to create emotionally resonant environments that foster connection and improve well-being.' |
| prompt | 'Generate a thought leadership article on the future of urban development regarding the "New Experiential Hybrid," a mixed-use approach that integrates commerce, culture, and leisure to address loneliness and build community in a rapidly changing world.' |

## Creating the embedding database with ChromaDB

Next we create a [custom function](https://docs.trychroma.com/guides/embeddings#custom-embedding-functions) to generate embeddings with the Gemini API. 

In our first step, we are implementing a retrieval system, so the `task_type` for generating the *document* embeddings is `retrieval_document`. Later, we will use `retrieval_query` for the *query* embeddings. More supported tasks: [API reference](https://ai.google.dev/api/embeddings#v1beta.TaskType) 

In [13]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.genai import types

class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True
    # Specify embedding model
    embedding_model = "models/text-embedding-004"

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model=self.embedding_model,
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

Now we create a [Chroma database client](https://docs.trychroma.com/getting-started) that uses the `GeminiEmbeddingFunction` and populate the database with the documents we loaded above, as well as linking to the correct metadata

In [14]:
import chromadb

DB_NAME = "genslerthoughtdb"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

# Delete by ID before re-adding
db.delete(ids=[str(i) for i in range(len(parsed_docs))])

db.add(
    documents=parsed_docs,
    metadatas=metadatas,
    ids=[str(i) for i in range(len(parsed_docs))]
)

Confirm that the data was inserted by looking at the database.

In [15]:
db.count()

20

In [16]:
# We can peek at the data too.

# Remove the 'document' field to only see the metadata and vectors first
clean_peek = {k: v for k, v in db.peek(1).items() if k != 'documents'}

pprint.pprint(clean_peek)

{'data': None,
 'embeddings': array([[ 2.60532126e-02, -1.67892277e-02,  1.00144623e-02,
        -2.77677197e-02, -1.22213010e-02,  1.37228826e-02,
         2.02265177e-02, -1.13578211e-03, -3.36452038e-03,
         3.03815361e-02, -1.22409957e-02,  7.27915764e-02,
         8.02195072e-02,  1.10239536e-02, -2.01436505e-02,
        -6.99245930e-02,  3.78048010e-02, -2.24709269e-02,
        -1.15451403e-01, -5.27875088e-02,  1.86103899e-02,
         1.02146203e-02, -7.93299172e-03, -8.12988058e-02,
        -3.12062204e-02, -2.04175338e-02, -7.52075016e-03,
         3.91868018e-02, -2.32383255e-02, -4.87098005e-03,
         2.47748829e-02,  5.09437732e-03,  1.55769866e-02,
         6.12767637e-02,  3.39666451e-03, -2.40138453e-02,
        -2.21656356e-02, -6.16950132e-02, -3.87425092e-03,
        -3.62799577e-02,  1.89356431e-02,  3.16973180e-02,
        -1.07421251e-02,  5.21871634e-02, -1.90407485e-02,
        -4.58655059e-02,  1.65937971e-02,  3.90238687e-02,
        -6.83777779e-02,  

In [17]:
# Peaking at the 1st document in the databse
Markdown(db.peek(1)['documents'][0])

| Metadata | Value |
|-----|-------|
| title | 'The New Experiential Hybrid' |
| source | 'https://www.gensler.com/blog/the-new-experiential-hybrid' |
| author | ['[[Gensler]]'] |
| published | None |
| created | datetime.date(2025, 4, 9) |
| description | 'Gensler is a global architecture, design, and planning firm with 57 offices and 6,000+ professionals across the Americas, Europe, Greater China, and APME.' |
| tags | ['clippings'] |

# The New Experiential Hybrid
![A street with cars and buildings.](https://static2.gensler.com/uploads/image/96476/Sportsmens_Lodge_N7_1738969737.jpg)

The Sportsmen’s Lodge, Los Angeles, California

The convergence of our physical and digital ecosystems continues to revolutionize our lives, reshape our behaviors, and reimagine our ability to navigate the complex beauty and challenges of life in a boundaryless world. Radical technological innovations have resulted in a fundamental mindset shift of our behaviors, needs, and desires, moving from Live, Work, Play (and Shop), to a hybrid of **Live, Live, Live**.

While many of us feel Covid-19 is largely in the rearview mirror, unfortunately a tapestry of global social epidemics still affects us all. Americans are lonelier than ever: [according to the Cigna U.S. Loneliness Index](https://newsroom.thecignagroup.com/loneliness-epidemic-persists-post-pandemic-look), 58% of adults, and 79% of those aged 18-24 report being lonely. Our social networks are a third smaller now and we’re far more likely to live alone; today, nearly 28% of U.S. homes have just one occupant, up from 17% in 1969 according to the US Census. The annual number of marriages is in decline, and birth rates are slowing. In a lonelier, more divided world, the need for community is greater than ever.

There’s a lot to consider with this cascade of change occurring at the speed of culture. As landlords, developers, and designers collaborate for a better future, we need to start thinking differently. It is not enough to simply evolve the concept of our built environment. Since we now live a hybridized life, we need to make a transformative shift to a “New Hybrid” experience, one that can play a significant role in restoring our sense of closeness in our communities. The New Hybrid experience is a model that uses an integrative approach to the disciplines of art, science, and technology, helping us to develop a deeper understanding of the complexities of our modern world and guide us to produce positive and impactful outcomes.

The New Hybrid experience brings together the robust attributes of commerce, culture, and leisure, and recalibrates the urban experience. It is amenity rich, leverages technology, connects easily to transit, and offers world-class leisure and event activities. It promotes local artisans and culture, reflects generational and community values, and builds traditions.

We relate to urban environments in an emotional dimension. Signature characteristics of hybrid places — otherwise called mixed use — are personable and human-centric. The value proposition is when a space, a place, a building, or an event **moves** us. When this happens, it elicits a sense of awe and wonder, engages all of our senses, dissolves barriers, and eliminates pain points.

The emotional exchange between the built environment and the user is about attaining and retaining a person’s attention. It’s that visceral moment when something inside you connects with the place in a special way, the beyond ordinary life experience that can elicit an emotional response that requires no specialized knowledge, nothing more than one’s emotion to make it happen. And it happens to everyone. Great places have a defined persona, which has the ability to trigger aesthetic emotions — that highly emotive sense you get, which can sometimes be difficult to describe, but you just know it just feels amazing.

Here are some examples of how the New Hybrid is playing out in our project work across multiple scales:

For [The Hub on Causeway](https://www.gensler.com/projects/the-hub-on-causeway), a dynamic, 2-million-square-foot mixed-use development at Boston’s TD Garden, Gensler created a marquee sports and entertainment destination with office, residential, hotel, retail, entertainment, dining, and creative office space that culminates in a grand entrance that funnels people to TD Garden and the North Station transit hub.

![A street with people and buildings along it.](https://static2.gensler.com/uploads/image/96477/HubonCauseway_N38_1738969781.jpg)

The Hub on Causeway, Boston, Massachusetts. Photo by Anton Grassl.

On a smaller scale, Gensler transformed a former meeting and convention space into the Sportsmen’s Lodge, an upscale outdoor lifestyle center in LA’s Studio City with 95,700-square-feet of walkable retail, restaurants, fitness, and public space. An open-air plaza features communal spaces for visitors to dine or relax, while a waterfront deck hosts tenant and center events.

![People walking on a sidewalk.](https://static2.gensler.com/uploads/image/96478/Sportsmens_Lodge_N4_1738969821.jpg)

The Sportsmen’s Lodge, Los Angeles, California

**Designed for maximum happiness, the New Hybrid is a magnetic and transformational place that transcends the predictable**. An amazing hybrid experience creates cultural capital by truly caring for us and improves the way we are. If our lives are truly enriched, the place will be more than real estate — it creates a deep, emotional connection with us and truly becomes an integral part of our lives.

For media inquiries, email [media@gensler.com](https://www.gensler.com/blog/).

## Retrieval: Find relevant documents

To search the Chroma database, we will call the `query` method. Note that we also switch to the `retrieval_query` mode of embedding generation, so the query will be embeded for the purpose of query to the databse.


In [18]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "Provide me 5 issues to think about when designing for return to office space."

top_k = 5

result = db.query(query_texts=[query], n_results=top_k)
#[all_articles] = result["documents"]

for article in result["documents"][0]:
    display(Markdown(article[:2000]))
    display(Markdown('---'))

| Metadata | Value |
|-----|-------|
| title | '3 Surprises Impacting the Return to the Office' |
| source | 'https://www.gensler.com/blog/3-surprises-impacting-the-return-to-the-office' |
| author | ['[[Gensler]]'] |
| published | None |
| created | datetime.date(2025, 4, 9) |
| description | 'Gensler is a global architecture, design, and planning firm with 57 offices and 6,000+ professionals across the Americas, Europe, Greater China, and APME.' |
| tags | ['clippings'] |

# 3 Surprises Impacting the Return to the Office
![A group of people sitting on couches in a room with large windows.](https://static1.gensler.com/uploads/hero_element/21205/thumb_tablet/thumbs/McCann_N15_1675895087_1024.jpg)

A group of people sitting on couches in a room with large windows.

*Editor’s Note: This blog is part of our [blog series](https://www.gensler.com/blog/by-keyword?k=WPS22) exploring insights from Gensler’s 2022 Workplace Survey findings.*

Research often measures and confirms what we, as designers and strategists, may intuitively believe based on our experience and practice. As a researcher, I love uncovering surprises in the data that debunk common perceptions. These surprising truths often lead to a new understanding of how people work and what they value, as well as new insights on the built environment. In [our latest workplace research](https://www.gensler.com/gri/us-workplace-survey-2022) study, we saw a number of unexpected surprises!

Here are the top three surprises uncovered in our research impacting the return to the office:

## Surprise #1: A Shift in the Role of the Office

Gensler conducted 11 different workplace research studies during the pandemic to understand how work and employee expectations were changing and what it meant for the new role of the office. In our surveys, we asked office workers and senior leaders to choose their top five most important reasons to come into the office. Throughout the pandemic the top-ranked reason was to “work in-person w

---

| Metadata | Value |
|-----|-------|
| title | 'Repopulating the Workplace: Why Collaboration Space Is in Danger' |
| source | 'https://www.gensler.com/blog/repopulating-the-workplace-collaboration-space-in-danger' |
| author | ['[[Gensler]]'] |
| published | None |
| created | datetime.date(2025, 4, 9) |
| description | 'Gensler is a global architecture, design, and planning firm with 57 offices and 6,000+ professionals across the Americas, Europe, Greater China, and APME.' |
| tags | ['clippings'] |

# Repopulating the Workplace: Why Collaboration Space Is in Danger
![A group of people in a room.](https://static2.gensler.com/uploads/image/96555/EdelmanLA2023_GobutyRyan_07_1739474705.jpg)

Edelman Los Angeles. Photo by Ryan Gobuty.

*Editor’s note: This article was originally published in [Work Design Magazine](https://www.workdesign.com/2025/02/informal-collaboration-space/).*

In the aftermath of the pandemic and the resulting shift to remote and hybrid work models, many organizations have reduced their real estate footprint, in some cases, significantly. Today, as more and more companies look to increase office presence for employees, they are having to rethink their workplace strategies to accommodate a greater daily population.

They’re grappling with whether to lease more space or repurpose shared space to accommodate increased demand for workstations and offices. This dilemma is a microcosm of broader organizational challenges that often pit financial considerations against employee needs and cultural implications.

## Why Collaboration Space Increased as Footprints Decreased

The pandemic prompted a paradigm shift in how offices are used. The purpose of the office was less about individual work and more about interactive work. Traditional cubicles gave way to more open, collaboration-focused layouts with minimal shared individual space types. Meeting rooms increased in number and variety, and breakout spaces, huddle rooms, and lounge areas became the focal 

---

| Metadata | Value |
|-----|-------|
| title | 'The Future Is Mixed Use: How Principles of Mixed Use Design Will Restore Our Communities' |
| source | 'https://www.gensler.com/blog/how-principles-of-mixed-use-design-will-restore-communities' |
| author | ['[[Gensler]]'] |
| published | None |
| created | datetime.date(2025, 4, 9) |
| description | 'Gensler is a global architecture, design, and planning firm with 57 offices and 6,000+ professionals across the Americas, Europe, Greater China, and APME.' |
| tags | ['clippings'] |

# The Future Is Mixed Use: How Principles of Mixed Use Design Will Restore Our Communities
![A high angle view of a building.](https://static2.gensler.com/uploads/image/96187/Hawkins_The_Line_N15_1738016965.jpg)

The Line Charlotte, Charlotte, North Carolina. Photo by Chad Mellon.

The real estate industry is in a time of great uncertainty with volatile inflation and interest rates, rising construction costs, major geopolitical events, and social and economic futures that feel unpredictable. Similar to how the past several years created opportunities to fundamentally rethink how we live, work, and interact with our communities and the built environment, the current upheaval is a chance for us to continue to adapt and innovate.

Amid all the ambiguity, the one type of development that has been thriving is mixed-use environments, which are key to reviving our communities’ urban centers. As traditional single-use spaces like office parks and retail centers struggle to get back to pre-pandemic occupancy rates, mixed-use projects present more optimal returns on investment for developers, since they lease for higher values and are more flexible to pivot during economic downturns.

Mixed-use environments also play an integral role in the revitalization of our cities because they are designed to be experience driven. By serving diverse populations and a wider range of functions, they create dynamic and inclusive spaces that people want to be part of

---

| Metadata | Value |
|-----|-------|
| title | 'Trends to Watch Reshaping the Future of Cities and Urban Living' |
| source | 'https://www.gensler.com/blog/trends-reshaping-future-of-cities-urban-living' |
| author | ['[[Gensler]]'] |
| published | None |
| created | datetime.date(2025, 4, 9) |
| description | 'Gensler is a global architecture, design, and planning firm with 57 offices and 6,000+ professionals across the Americas, Europe, Greater China, and APME.' |
| tags | ['clippings'] |

# Trends to Watch Reshaping the Future of Cities and Urban Living
![A busy street with people and cars.](https://static2.gensler.com/uploads/hero_element/23551/thumb_tablet/thumbs/Fleet-Street-area-of-opportunity_1710365013_1024.jpg)

A busy street with people and cars.

*Editor’s Note: This blog is part of our [Design Forecast blog series](https://www.gensler.com/blog/by-keyword?k=Design_Forecast_2024), looking at what’s next in 2024 and beyond. We sat down with [Andre Brumfield](https://www.gensler.com/people/andre-brumfield), [Ian Mulcahey](https://www.gensler.com/people/ian-mulcahey), and [Chris Rzomp](https://www.gensler.com/people/christopher-rzomp), global leaders of Gensler’s [Cities & Urban Design practice](https://www.gensler.com/expertise/cities-urban-design), to discuss what’s next for the future of cities.*

## Many downtowns are struggling in the wake of the pandemic, and clients want to know how to invest effectively. How are you advising them? And how long do you think it will take for cities to fully bounce back from the pandemic?

**Andre Brumfield:** It’s going to be a process that’s going to unfold for some cities in the next five years, while others may take a decade or generation to do so. Downtowns have died and come back for centuries. In the U.S., that’s certainly been the case and cities have always evolved into something else. We have to accept that our downtowns are going to be something different. They cannot be monochromatic in terms of being domina

---

| Metadata | Value |
|-----|-------|
| title | 'Avoiding Fear-Based Decision-Making to Foster Positive In-Store Retail Experiences' |
| source | 'https://www.gensler.com/blog/avoiding-fear-based-decision-making-retail-experiences' |
| author | ['[[Gensler]]'] |
| published | None |
| created | datetime.date(2025, 4, 9) |
| description | 'Gensler is a global architecture, design, and planning firm with 57 offices and 6,000+ professionals across the Americas, Europe, Greater China, and APME.' |
| tags | ['clippings'] |

# Avoiding Fear-Based Decision-Making to Foster Positive In-Store Retail Experiences
![A person sitting in a chair.](https://static2.gensler.com/uploads/image/95670/Squint-Eyewear_N4_2000x1125_1734717930.jpg)

Squint Eyewear, Toronto. Photo by Ben Rahn/A-Frame.

Over the course of my 30-year long career as a retail designer, I’ve seen more than my fair share of ups-and-downs. Numerous recessions, a pandemic, and the birth of one key disruption that is now long considered unimaginable to live without. Like in fashion, trends come and go, but the recurring notion that in-store shopping is dead has continued to plague brick-and-mortar retail since the inception of the e-tail — a headline that’s as played out as the “Bedazzler.” In reality, in-store shopping is alive and well, but not all retail experiences are created equal.

Despite the convenience of online shopping, there is something irreplaceable about the in-store experience. Only in-store can customers compare fabrics, try on for size, shop socially, or simply have the opportunity to stumble upon something new or unexpected. While there are exceptions, algorithms usually don’t lead shoppers to an adrenaline rush from stumbling upon a great discovery! But while in-store shopping is inimitable in the digital realm, it hinges on its ability to accurately reflect a brand, connect with customers, and offer a seamless journey through space, and it starts with deliberate, bespoke, and often bold design de

---

In [19]:
# check the full result with metadata
pprint.pprint(result)

{'data': None,
 'distances': [[0.7353310585021973,
                0.7428976893424988,
                0.8130573034286499,
                0.8715016841888428,
                0.8874541521072388]],
 'documents': [['| Metadata | Value |\n'
                '|-----|-------|\n'
                "| title | '3 Surprises Impacting the Return to the Office' |\n"
                '| source | '
                "'https://www.gensler.com/blog/3-surprises-impacting-the-return-to-the-office' "
                '|\n'
                "| author | ['[[Gensler]]'] |\n"
                '| published | None |\n'
                '| created | datetime.date(2025, 4, 9) |\n'
                "| description | 'Gensler is a global architecture, design, "
                'and planning firm with 57 offices and 6,000+ professionals '
                "across the Americas, Europe, Greater China, and APME.' |\n"
                "| tags | ['clippings'] |\n"
                '\n'
                '# 3 Surprises Impacting the Re

## Augmented generation: Answer the question

Now that you have found several relevant passage from the set of documents (the *retrieval* step), we can now assemble a generation prompt to have the Gemini API *generate* a final answer. In practice, when the size of underlying data is large, we will want to retrieve more than one result and let the Gemini model determine what passages are relevant in answering the question. For this reason it's OK if some retrieved passages are not directly related to the question - this generation step should ignore them.

### Citation

Ideally we would also like to structure the output to link parts of the answer to the retrival given, to make it easier to fact check and put the answer in context, in order to do that, we should include the retrivals into the prompt in a structured way including ids for ease of indexing and matching, as well as metadatas like title, source links, etc. to be able to build a citation footnote as part of the response.

In [24]:
import json

structured_retrival = {
    "articles": []
}


for i in range(len(result["documents"][0])):
    doc = result["documents"][0][i]
    meta = result["metadatas"][0][i]
    dist = result["distances"][0][i]
    
    structured_retrival["articles"].append({
        "ref_id": i + 1,
        "title": meta.get("title", "Untitled"),
        "url": meta.get("source", ""),
        "distance": dist,
        "content": doc
    })

# articles to one line   
article_refs_str = "\n---\n".join(
    f"[{a['ref_id']}] {a['title']}\n{a['content']}." 
    for a in structured_retrival["articles"]
)

# articles to json
article_refs_json = json.dumps(structured_retrival["articles"], indent=2)

# question to one line
question = query.replace("\n", " "),

# overal guidance for the response
prompt_context = f"""
You are a helpful knowledge management assitant with access to a proprietary database.
You will answers questions using articles retrieved from the database listed below. 
Be sure to respond in complete sentences, being comprehensive, including all relevant background information, and dive into the details when necessary.
Be sure to break down complicated concepts and strike a friendly and converstional tone. 
If the passage is irrelevant to the answer, you may ignore it.

QUESTION:
{question}

ARTICLES:
{article_refs_json}

OUTPUT FORMAT:
- Provide a detailed, well-written answer.
- Cite sources inline using [ref_id] (e.g. [1], [2]) at the end of sentences or paragraphs.
- Only cite articles that were actually used.
- At the end, return your answer and the list of used citation ref_ids in JSON format like:
  {{
    "answer": "Your answer text here [1][2]...",
    "citations": [1, 2]
  }}
"""

print(prompt_context)
Markdown('---')


You are a helpful knowledge management assitant with access to a proprietary database.
You will answers questions using articles retrieved from the database listed below. 
Be sure to respond in complete sentences, being comprehensive, including all relevant background information, and dive into the details when necessary.
Be sure to break down complicated concepts and strike a friendly and converstional tone. 
If the passage is irrelevant to the answer, you may ignore it.

QUESTION:
('Provide me 5 issues to think about when designing for return to office space.',)

ARTICLES:
[
  {
    "ref_id": 1,
    "title": "3 Surprises Impacting the Return to the Office",
    "url": "https://www.gensler.com/blog/3-surprises-impacting-the-return-to-the-office",
    "distance": 0.7353310585021973,
    "content": "| Metadata | Value |\n|-----|-------|\n| title | '3 Surprises Impacting the Return to the Office' |\n| source | 'https://www.gensler.com/blog/3-surprises-impacting-the-return-to-the-office'

---

### Structured Output

Now to be able to make parsing the response and building the footnote eaiser and less error-prone, we can structured the output into `json` format in order to assign each type of infomation to variables, upon which we can reformat after response is given from the mode.

We start by first create a `response_schema` class and assigne the response format to be `application/json`.

In [27]:
from typing import List
from typing_extensions import TypedDict

class AnswerWithCitations(TypedDict):
    answer: str
    citations: List[int]

structured_config = types.GenerateContentConfig(
    response_mime_type="application/json",
    response_schema=AnswerWithCitations,
    temperature=0.1
)

Now we use the `generate_content` method to to generate an answer to the question, while following the structured prompt and output format.

We would also parse the returned `json` file and reformat it as desired, building a nice citation footnote at the end.

In [30]:
# load the prompt which includes the system instruction, retrival, citation, etc.
# load the config that specified return format
response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=structured_config,
    contents=[prompt_context] 
)

structured = response.parsed # response parsed into the requested json format

# Build footnote section from actually used citations
used_ids = set(structured["citations"])

footnotes = "\n".join(
    f"[{a['ref_id']}] {a['title']} [Link]({a['url']}) | Relevance: {(1 - a['distance']) * 100:.1f}%\n"
    for a in structured_retrival["articles"]
    if a["ref_id"] in used_ids
)

final_output = f"{structured['answer']}\n\n---\n**References**\n\n{footnotes}"
display(Markdown(final_output))

When designing for a return to the office space, it's important to consider several key issues to create an environment that meets the evolving needs of employees and organizations. Here are five issues to think about:

1.  **The Shifting Role of the Office:** Understand that the primary reason people come to the office has shifted from in-person collaboration to focusing on individual work [1]. Employees now value the office for access to technology, specific spaces, materials, and resources that help them get their work done [1]. Therefore, the design should prioritize spaces that support focused work, while still providing opportunities for collaboration.
2.  **The Need for More Office Time:** Recognize that employees feel they need to spend more time in the office to maximize both individual and team productivity [1]. Innovative companies report needing employees in the office about 67% of the work week [1]. Understanding why employees aren’t coming into the office despite this need is crucial. Addressing what works and what doesn’t in the current office environment can help bridge this gap [1].
3.  **The Importance of Collaboration Spaces:** Despite the shift towards individual focus, collaboration remains a critical aspect of office work [2]. Companies want employees back in the office to increase collaboration, interaction, and learning [2]. Repurposing collaboration spaces into individual workstations can be counterproductive, frustrating employees and undermining the goal of increased in-person collaboration [2].
4.  **The Blend of Individual and Collaborative Work:** Embrace a "both/and" approach that supports both individual focus work and collaborative work [2]. The design should not sacrifice one for the other but should aim to create an environment that excels in both areas [2]. This approach will lead to better experiences, stronger cultures, and positive business outcomes [2].
5.  **The Need for Adaptability and Mixed-Use Principles:** Design for resilience by creating environments suited to changing work and life patterns [3]. Mixed-use environments that incorporate a variety of experiences and functions can make the office a more attractive destination [3]. Consider incorporating elements that encourage 24/7 activation, such as community spaces, to make the office feel like an extension of the neighborhood [3].

---
**References**

[1] 3 Surprises Impacting the Return to the Office [Link](https://www.gensler.com/blog/3-surprises-impacting-the-return-to-the-office) | Relevance: 26.5%

[2] Repopulating the Workplace: Why Collaboration Space Is in Danger [Link](https://www.gensler.com/blog/repopulating-the-workplace-collaboration-space-in-danger) | Relevance: 25.7%

[3] The Future Is Mixed Use: How Principles of Mixed Use Design Will Restore Our Communities [Link](https://www.gensler.com/blog/how-principles-of-mixed-use-design-will-restore-communities) | Relevance: 18.7%
